In [3]:
from scrapfly import ScrapflyClient, ScrapeConfig, ScrapflyError
from jsonpath_ng import jsonpath, parse
 
scrapfly = ScrapflyClient(key="scp-live-01fc8534987042f4a2fe553c5cf6a2df", max_concurrency=2)
url = 'https://www.nfm.com/canon-eos-r50-rf-s18-45mm-f4-5-6-3-is-stm-lens-kit-black-mirrorless-camera-64075492/64075492.html'
listing_result = scrapfly.scrape(ScrapeConfig(url=url, render_js=True, country="US", asp=True, retry=False, rendering_wait=10000))
listing_html = listing_result.scrape_result['content']

In [4]:
from bs4 import BeautifulSoup
import re
import json
def extract_json_from_html(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    script_tags = soup.find_all('script')

    json_data_list = []

    for script in script_tags:
        script_content = script.string
        if script_content:
            # Use a regular expression to find JSON-like content
            json_pattern = re.compile(r'({.*})', re.DOTALL)
            matches = json_pattern.findall(script_content)
            for match in matches:
                try:
                    json_data = json.loads(match)
                    json_data_list.append(json_data)
                except json.JSONDecodeError:
                    pass  # Ignore invalid JSON

    return json_data_list

json_data_list = extract_json_from_html(listing_html)

In [17]:
json_data_list

[{},
 {},
 {'@context': 'http://schema.org/',
  '@type': 'Product',
  'name': 'Canon EOS R50 RF-S18-45mm F4.5-6.3 IS STM Lens Kit (Black) Mirrorless Camera',
  'mpn': '5811C012',
  'sku': '64075492',
  'description': '\n24.2 Megapixel CMOS (APS-C) sensor\n4K uncropped movie at up to 30 fps oversampled from 6K\nDual Pixel CMOS AF II\nMovie for Close-up Demo Mode\n12 fps Electronic First Curtain and 15 fps Electronic Shutter Continuous shooting\nAdvanced A+ Assist with expanded array of auto-compatible scenes\nLonger than 30min video recording\nUVC/UAC compatibility for webcam and streaming\n\nIn the Box\n\nEOS R50 Body (BK)\nCamera Cover R-F-5\nNeck Strap EM-200DB\nBattery Charger LC-E17\nBattery Pack LP-E17\nBattery Pack Cover\nRF-S18-45mm F4.5-6.3 IS STM\nLens Cap E-49\nLens Dust Cap\nRF Shoe Cover\n \n',
  'aggregateRating': {'@type': 'AggregateRating',
   'ratingValue': 4,
   'reviewCount': 14},
  'brand': {'@type': 'Brand', 'name': 'Canon'},
  'gtin12': '013803351415',
  'image': [

In [21]:
from jsonpath_ng import jsonpath, parse
paths = {
    'productName': "$..name",
    'mpn': "$..mpn",
    'sku': "$..sku",
    'description': "$..description",
    "rating" :"$..aggregateRating.ratingValue",
    "brand" :"$..brand.name",
    "upc" :"$..gtin12",
    "image" :"$..image",
    'currency': "$..offers.priceCurrency",
    'price': "$..offers.price",  # Example for price
    'availability': "$..offers.availability",
    'productId': "$..@id"
}

In [19]:
extracted_data = {}
for key, path in paths.items():
    jsonpath_expr = parse(path)
    matches = jsonpath_expr.find(json_data_list)
    extracted_data[key] = [match.value for match in matches]

In [20]:
extracted_data

{'productName': ['Canon EOS R50 RF-S18-45mm F4.5-6.3 IS STM Lens Kit (Black) Mirrorless Camera',
  'Canon'],
 'mpn': ['5811C012'],
 'sku': ['64075492'],
 'description': ['\n24.2 Megapixel CMOS (APS-C) sensor\n4K uncropped movie at up to 30 fps oversampled from 6K\nDual Pixel CMOS AF II\nMovie for Close-up Demo Mode\n12 fps Electronic First Curtain and 15 fps Electronic Shutter Continuous shooting\nAdvanced A+ Assist with expanded array of auto-compatible scenes\nLonger than 30min video recording\nUVC/UAC compatibility for webcam and streaming\n\nIn the Box\n\nEOS R50 Body (BK)\nCamera Cover R-F-5\nNeck Strap EM-200DB\nBattery Charger LC-E17\nBattery Pack LP-E17\nBattery Pack Cover\nRF-S18-45mm F4.5-6.3 IS STM\nLens Cap E-49\nLens Dust Cap\nRF Shoe Cover\n \n'],
 'rating': [4],
 'brand': ['Canon'],
 'upc': ['013803351415'],
 'image': [['https://www.nfm.com/dw/image/v2/BDFM_PRD/on/demandware.static/-/Sites-nfm-master-catalog/default/dw761964be/images/064/07/64075492-1.jpg?sw=1000&sh=1000